# Challenge 2

In this challenge we will continue working with the `Pokemon` dataset. We will attempt solving a slightly more complex problem in which we will practice the iterative data analysis process you leaned in [this video](https://www.youtube.com/watch?v=xOomNicqbkk).

The problem statement is as follows:

**You are at a Pokemon black market planning to buy a Pokemon for battle. All Pokemon are sold at the same price and you can only afford to buy one. You cannot choose which specific Pokemon to buy. However, you can specify the type of the Pokemon - one type that exists in either `Type 1` or `Type 2`. Which type should you choose in order to maximize your chance of receiving a good Pokemon?**

To remind you about the 3 steps of iterative data analysis, they are:

1. Setting Expectations
1. Collecting Information
1. Reacting to Data / Revising Expectations

Following the iterative process, we'll guide you in completing the challenge.


## Problem Solving Iteration 1

In this iteration we'll analyze the problem and identify the breakthrough. The original question statement is kind of vague because we don't know what a *good pokemon* really means as represented in the data. We'll start by understanding the dataset and see if we can find some insights.

In [ ]:
####LIBRARIES
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import pymysql


# import data set from Ironhack's database


In [ ]:

##CONNECTION
driver = 'mysql+pymysql'
ip = '34.65.10.136'
username = 'data-students'
password = 'iR0nH@cK-D4T4B4S3'
db = 'pokemon'
connection_string  = f'{driver}://{username}:{password}@{ip}/{db}'
engine = create_engine(connection_string)



#  WHICH TABLES WE WANT TO CHOOSE    


In [ ]:
query = 'SELECT * from pokemon_stats'
pokemon = pd.read_sql(query,engine)

pokemon.head()

From the data it seems whether a pokemon is good depends on its abilities as represented in the fields of `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, `Speed`, and `Total`. We are not sure about `Generation` and `Legendary` because they are not necessarily the decisive factors of the pokemon abilities.

But `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, `Speed`, and `Total` are a lot of fields! If we look at them all at once it's very complicated. This isn't Mission Impossible but it's ideal that we tackle this kind of problem after we learn Machine Learning (which you will do in Module 3). For now, is there a way to consolidate the fields we need to look into?

Fortunately there seems to be a way. It appears the `Total` field is computed based on the other 6 fields. But we need to prove our theory. If we can approve there is a formula to compute `Total` based on the other 6 abilities, we only need to look into `Total`.

We have the following expectation now:

#### The `Total` field is computed based on `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, and `Speed`.

We need to collect the following information:

* **What is the formula to compute `Total`?**
* **Does the formula work for all pokemon?**

In the cell below, make a hypothesis on how `Total` is computed and test your hypothesis.

 MY HYPOTHESIS IS THIS:

In [ ]:
Hypothesis = pokemon['HP']+pokemon['Attack']+pokemon['Defense']+pokemon['Sp. Atk']+pokemon['Sp. Def']+pokemon['Speed']

check if the all values of TOTAL obbeys my hypothesis

In [ ]:
all (pokemon['Total'] == Hypothesis)

## Problem Solving Iteration 2

Now that we have consolidated the abilities fields, we can update the problem statement. The new problem statement is:

### Which pokemon type is most likely to have the highest `Total` value?

In the updated problem statement, we assume there is a certain relationship between the `Total` and the pokemon type. But we have two *type* fields (`Type 1` and `Type 2`) that have string values. In data analysis, string fields have to be transformed to numerical format in order to be analyzed. 

In addition, keep in mind that `Type 1` always has a value but `Type 2` is sometimes empty (having the `NaN` value). Also, the pokemon type we choose may be either in `Type 1` or `Type 2`.

Now our expectation is:

#### `Type 1` and `Type 2` string variables need to be converted to numerical variables in order to identify the relationship between `Total` and the pokemon type.

The information we need to collect is:

#### How to convert two string variables to numerical?

Let's address the first question first. You can use a method called **One Hot Encoding** which is frequently used in machine learning to encode categorical string variables to numerical. The idea is to gather all the possible string values in a categorical field and create a numerical field for each unique string value. Each of those numerical fields uses `1` and `0` to indicate whether the data record has the corresponding categorical value. A detailed explanation of One Hot Encoding can be found in [this article](https://hackernoon.com/what-is-one-hot-encoding-why-and-when-do-you-have-to-use-it-e3c6186d008f). You will formally learn it in Module 3.

For instance, if a pokemon has `Type 1` as `Poison` and `Type 2` as `Fire`, then its `Poison` and `Fire` fields are `1` whereas all other fields are `0`. If a pokemon has `Type 1` as `Water` and `Type 2` as `NaN`, then its `Water` field is `1` whereas all other fields are `0`.

#### In the next cell, use One Hot Encoding to encode `Type 1` and `Type 2`. Use the pokemon type values as the names of the numerical fields you create.

The new numerical variables you create should look like below:

![One Hot Encoding](../images/one-hot-encoding.png)


#   ENCODE STRING VARIABLES (letters) in numbers to make computation.


we use the get_dummies funcion from pandas to specify which columns from pokemon we want to encode.
the prefix and prefix_sep we use to remove prename that appears by default


In [ ]:
encoding = pd.get_dummies(pokemon[['Type 1', 'Type 2']],prefix = '', prefix_sep='')
encoding.head()

As we have all pokemon types repeated in 'Types 1'  and 'Types 2' columns, we group by columns. 

In [ ]:
encoding_sum = encoding.groupby(encoding.columns, axis =1).sum()
encoding_sum.head()


## Problem Solving Iteration 3

Now we have encoded the pokemon types, we will identify the relationship between `Total` and the encoded fields. Our expectation is:

#### There are relationships between `Total` and the encoded pokemon type variables and we need to identify the correlations.

The information we need to collect is:

#### How to identify the relationship between `Total` and the encoded pokemon type fields?

There are multiple ways to answer this question. The easiest way is to use correlation. In the cell below, calculate the correlation of `Total` to each of the encoded fields. Rank the correlations and identify the #1 pokemon type that is most likely to have the highest `Total`.

Creation a new table with the encoding of type 1 and type 2 grouped by the sum of Total 

In [ ]:
#Addition column total to the table where we create the encoding
encoding_sum['Total'] = pokemon['Total']
encoding_total = encoding_sum.groupby(encoding_sum.Total).sum()
encoding_total.head()

Create the correlation

In [ ]:

correlation = encoding_total.corr(method = 'pearson')
correlation

To choose the more 'powerful' type pokemon we remove the correlation with same type, because a correlation with itself is one.

We look foor the max value for each type: **value_type**

we search the types which coincides with these max value: **type**




In [ ]:
top1 = correlation[(correlation <1)]    ## correlation table removing elements equal to 1 (same type correlation = 1)


type_pokemon = list(top1.idxmax())         ## types of pokemon that has the max corr for each column


value_type = top1.max().unique()        ##values of corr. for each kind of pokemon for each column.We choose the non-repeated ones.


lista =list(zip(top1.columns,type_pokemon, value_type))  ##creation a list with the combination of type1 (rows), 
                                                         #and type 2 (columns) and their correlation values

print(lista)

In [ ]:
table = pd.DataFrame(lista)

table.rename(columns = {0:'Type 1',1:'Type 2', 2:'Correlation value'}, inplace = True)
table #order for each columns


table.sort_values('Correlation value', ascending = False,inplace = True)   #order for correlation value
table.reset_index(drop=True, inplace=True)   #reset the index.
table 

# Bonus Question

Say now you can choose both `Type 1` and `Type 2` of the pokemon. In order to receive the best pokemon, which types will you choose?

We use the previous table and see what's the type who has more correlation value.

In [ ]:
table

**The best option is to get a Dragon-Psychic pokemon**